In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Coding a simple model

In [ ]:
class MLP:
    def __init__(self, neurons_in_layer, learning_rate):
        self.neurons_in_layer = neurons_in_layer   # 1 input layer, 2 hidden, 1 output layer so a 1x4 array
        self.learning_rate = learning_rate
        
        self.layers = len(neurons_in_layer) # Excluding final layer

        # Initializing weights & biases
        self.weights = []
        self.biases = []
        

        for i in range(self.layers - 1):
            # In the matrices rows correspond to neurons in the current layer and columns to neurons in the next layer
            self.weights.append(np.random.uniform(-1,1,size=[neurons_in_layer[i],neurons_in_layer[i+1]]))
            self.biases.append(np.zeros((1, neurons_in_layer[i+1])))

        print(len(self.weights[0]))
        print(len(self.weights[1]))
        print(len(self.weights[2]))    
    def sigmoid(self, x):
        return 1./(1.+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        sigmoid_x = self.sigmoid(x)
        return sigmoid_x * (1 - sigmoid_x)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_derivative(self, x):
        return (x>0)*1
    
    # To have probabilities (all sum up to 1) in the output layer
    def softmax(self, x):
        print("x:", x)
        exp_x = np.exp(x)
        print("exp_x:", exp_x)
        prob = exp_x / exp_x.sum(axis=0, keepdims=True)
        print("prob:", prob)
        return prob
    
    def feed_forward(self, batch):
        self.layer_output = [batch]

        # Passing through each layer except the last one
        for i in range(self.layers - 1):
            output = self.layer_output[-1].dot(self.weights[i]) + self.biases[i]   # weight * x + bias
            self.layer_output.append(self.ReLU(output))

        # Using softmax in the output layer
        self.layer_output[-1] = self.softmax(self.layer_output[-1])
        
        print(self.layer_output[-1])
        return self.layer_output[-1]
        #final_x = np.dot(self.layer_output[-1], self.weights[-1]) + self.biases[-1]
        #self.x_values.append(final_x)
        #self.layer_output.append(self.softmax(final_x))

    def backpropagation(self, y, X_size):
        output_error = self.layer_output[-1] - y  # cross- entropy loss
        delta = output_error * self.ReLU_derivative(self.layer_output[-1])

        # Backpropagation through each layer backwards (duh)
        for i in range(1, self.layers - 1):
            self.weights[-i] -= self.learning_rate * (self.layer_output[-i-1].T.dot(delta))
            self.biases[-i] -= self.learning_rate * np.mean(delta, axis=0, keepdims=True)
            delta = self.ReLU_derivative(self.layer_output[-i-1]) * (delta.dot(self.weights[-i].T))
            # delta = deltas[-1].dot(self.weights[i+1].T) * self.ReLU_derivative(self.layer_output[i+1])
            # deltas.append(delta)

        # # Reversing deltas to match layer order
        # deltas.reverse()

        # # Updating weights and biases
        # for i in range(self.layers - 1):
        #     self.weights[i] -= self.learning_rate * self.layer_output[i].T.dot(deltas[i])
        #     self.biases[i] -= self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs = 1000):
        for epoch in range(epochs):
            output = self.feed_forward(X)
            self.backpropagation(y, len(X))
            #print(output)
            if epoch % 1 == 0:
                loss = -np.sum(y * np.log(output + 1e-9)) / X.shape[0]
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        output = self.feed_forward(X)
        print("Output:", output)
        print("Label:", np.argmax(output, axis=1))
        return np.argmax(output, axis=1)

    

In [ ]:
class MLP_new:
    def __init__(self, input, hidden_layer1, hidden_layer2, output, learning_rate, sample_size):
        self.input = input
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.output = output
        self.learning_rate = learning_rate
        
        self.sample_size = sample_size
        self.total_error = 0

        # Initializing weights and biases
        self.weights1 = np.random.uniform(-1, 1, size=[input, hidden_layer1])
        self.weights2 = np.random.uniform(-1, 1, size=[hidden_layer1, hidden_layer2])
        self.weights3 = np.random.uniform(-1, 1, size=[hidden_layer2, output])

        self.biases1 = np.zeros((1, hidden_layer1))
        self.biases2 = np.zeros((1, hidden_layer2))
        self.biases3 = np.zeros((1, output))

        self.final_output = []
        self.loss = []


    # Defining useful functions
    def sigmoid(self, x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        return 1/(1+np.exp(-x_shifted * 1.0))
        
    def sigmoid_prime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_prime(self,x):
        return (x>0)*1
    
    def softmax(self,x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        exp = np.exp(x_shifted)
        probabilities = exp / np.sum(exp, axis=1, keepdims=True)
        return probabilities
    
    def cross_entropy_loss(self, y):
        loss = 0
        # Loop through each sample
        for sample in range(y.shape[0]):
            # Get the predicted probabilities for the current sample
            probabilities = self.final_output[sample]
            
            # Ensure numerical stability by adding a small constant (epsilon) to avoid log(0)
            epsilon = 1e-9
            probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
            
            # Compute the loss for the current sample using the correct one-hot encoded label
            loss += -np.sum(y[sample] * np.log(probabilities))
        
        # Return the average loss
        return loss / y.shape[0]

    # Forward
    def train(self, X, y, epochs):
        self.loss = np.zeros([epochs,1])
        for epoch in range(epochs):
            self.final_output = np.zeros([self.sample_size, self.output])
            for sample, inputs in enumerate(X):
                inputs = inputs.reshape(-1,1).T
                # Forward pass
                output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
                output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
                output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

                self.final_output[sample] = output3

                # Backward pass
                error = output3 - y[sample]
                self.total_error += error
                
                delta_out = error
                self.weights3 -= self.learning_rate * (output2.T.dot(delta_out))
                self.biases3 -= self.learning_rate * delta_out
        
                delta2 = self.sigmoid_prime(output2) * (delta_out.dot(self.weights3.T))
                self.weights2 -= self.learning_rate * (output1.T.dot(delta2))
                self.biases2 -= self.learning_rate * delta2

                delta1 = self.sigmoid_prime(output1) * (delta2.dot(self.weights2.T))
                self.weights1 -= self.learning_rate * (inputs.T.dot(delta1))
                self.biases1 -= self.learning_rate * delta1

            self.total_error = self.total_error/len(y)

            self.loss[epoch] = self.cross_entropy_loss(y)
            if epoch % 10 == 0:
                print(f"Epoch: {epoch}, Loss: {self.loss[epoch]}, Final output: {self.final_output}")

    def predict(self, X):
        predictions = []
        self.final_output = np.zeros([self.sample_size, self.output])
        for sample, inputs in enumerate(X):
            # Forward pass
            inputs = inputs.reshape(-1,1).T
                # Forward pass
            output1 = self.sigmoid(inputs.dot(self.weights1) + self.biases1)
            output2 = self.sigmoid(output1.dot(self.weights2) + self.biases2)
            output3 = self.softmax(output2.dot(self.weights3) + self.biases3)

            self.final_output[sample] = output3

            predictions.append(np.argmax(self.final_output[sample]))
        return predictions

In [55]:
class MLP_new_batches:
    def __init__(self, input_size, hidden_layer1, hidden_layer2, output_size, learning_rate, batch_size):
        self.input_size = input_size
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.loss = []

        # Initialize weights and biases
        self.weights1 = np.random.uniform(-1, 1, size=(input_size, hidden_layer1))
        self.weights2 = np.random.uniform(-1, 1, size=(hidden_layer1, hidden_layer2))
        self.weights3 = np.random.uniform(-1, 1, size=(hidden_layer2, output_size))

        self.biases1 = np.zeros((1, hidden_layer1))
        self.biases2 = np.zeros((1, hidden_layer2))
        self.biases3 = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_prime(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def ReLU(self, x):
        return np.maximum(0, x)
    
    def ReLU_prime(self,x):
        return (x>0)*1

    def softmax(self, x):
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        exp = np.exp(x_shifted)
        return exp / np.sum(exp, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        epsilon = 1e-9
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

    def train(self, X, y, epochs):
        self.loss = np.zeros([epochs, 1])
        num_samples = X.shape[0]
        for epoch in range(epochs):
            # # Shuffle data
            # indices = np.arange(num_samples)
            # np.random.shuffle(indices)
            # X = X[indices]
            # y = y[indices]
            epoch_loss = 0
            for i in range(0, num_samples, self.batch_size):
                # Get batch
                X_batch = X[i:i + self.batch_size]
                y_batch = y[i:i + self.batch_size]

                # Forward pass
                z1 = X_batch.dot(self.weights1) + self.biases1
                a1 = self.ReLU(z1)

                z2 = a1.dot(self.weights2) + self.biases2
                a2 = self.ReLU(z2)

                z3 = a2.dot(self.weights3) + self.biases3
                a3 = self.softmax(z3)

                # Loss calculation
                batch_loss = self.cross_entropy_loss(a3, y_batch)
                epoch_loss += batch_loss

                # Backward pass
                error_out = a3 - y_batch
                grad_weights3 = a2.T.dot(error_out) / self.batch_size
                grad_biases3 = np.sum(error_out, axis=0, keepdims=True) / self.batch_size

                error_hidden2 = error_out.dot(self.weights3.T) * self.ReLU_prime(z2)
                grad_weights2 = a1.T.dot(error_hidden2) / self.batch_size
                grad_biases2 = np.sum(error_hidden2, axis=0, keepdims=True) / self.batch_size

                error_hidden1 = error_hidden2.dot(self.weights2.T) * self.ReLU_prime(z1)
                grad_weights1 = X_batch.T.dot(error_hidden1) / self.batch_size
                grad_biases1 = np.sum(error_hidden1, axis=0, keepdims=True) / self.batch_size

                # Update weights and biases
                self.weights3 -= self.learning_rate * grad_weights3
                self.biases3 -= self.learning_rate * grad_biases3

                self.weights2 -= self.learning_rate * grad_weights2
                self.biases2 -= self.learning_rate * grad_biases2

                self.weights1 -= self.learning_rate * grad_weights1
                self.biases1 -= self.learning_rate * grad_biases1

            self.loss[epoch] = epoch_loss / (num_samples / self.batch_size)

            if epoch % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / (num_samples / self.batch_size):.4f}")

    def predict(self, X):
        # Forward pass
        z1 = X.dot(self.weights1) + self.biases1
        a1 = self.ReLU(z1)

        z2 = a1.dot(self.weights2) + self.biases2
        a2 = self.ReLU(z2)

        z3 = a2.dot(self.weights3) + self.biases3
        a3 = self.softmax(z3)

        return np.argmax(a3, axis=1)

### Loading data

In [13]:
data_file = "C:/Users/afrod/Documents/Neural_Networks/MergedDataset"
classes = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]
training_data = []


def create_training_data():
    for dementia_level in classes:
        path = os.path.join(data_file, dementia_level)
        class_num = classes.index(dementia_level)
        for img in os.listdir(path):
            # Convert to grayscale for smaller array dimensions
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            final_array = cv2.resize(img_array, (100,95))
            training_data.append([final_array, class_num])

create_training_data()

In [14]:
random.shuffle(training_data)

# Separating features and labels
# Images are also flattened to be used as input in the knn algorithm
X = np.array([features for features, _ in training_data]).reshape(-1, 100*95)
y = np.array([label for _, label in training_data])

# Rescaling
X = (X-X.min())/(X.max() - X.min())

# One-hot encoding
y_onehot = np.zeros((y.size, int(y.max()) + 1))
y_onehot[np.arange(y.size),y.astype(int)] = 1.0

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.4, random_state=42)
print(X_train.shape[0])

24230


### Training time

In [ ]:
mlp = MLP_new(2350, 100, 100, 4, 0.001, len(X_train))

In [ ]:
mlp.train(X_train, y_train, 50)

In [58]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with layers [2350, 100, 100, 4], lr = 0.001, epochs = 50:")
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))

KeyboardInterrupt: 

In [ ]:
mlp.loss

In [27]:
loss_mat = np.matrix(mlp.loss)

with open('n_100_0.01_50.txt','wb') as f:
    for line in loss_mat:
        np.savetxt(f, line, fmt='%.5f')

In [21]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_train)
y_test_labels = np.argmax(y_train, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with layers [36100, 100, 100, 4], lr = 0.001, epochs = 50:")
print("Size: 100x95")
print(f'Train Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))


MLP with layers [36100, 100, 100, 4], lr = 0.001, epochs = 50:
Size: 100x95
Train Accuracy: 31.44%
Accuracy: 0.3143623607098638
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.47      0.41      7637
           1       0.29      0.29      0.29      6744
           2       0.31      0.26      0.28      5911
           3       0.21      0.13      0.16      3938

    accuracy                           0.31     24230
   macro avg       0.29      0.29      0.29     24230
weighted avg       0.30      0.31      0.30     24230

Confusion Matrix:
 [[3592 2226 1124  695]
 [2763 1972 1392  617]
 [2278 1471 1547  615]
 [1292 1204  936  506]]


In [56]:
mlp = MLP_new_batches(9500, 500, 500, 4, 0.001, 1000) #2350

In [57]:
mlp.train(X_train, y_train, 100)

Epoch 1/100, Loss: 15.2152
Epoch 11/100, Loss: 10.3104
Epoch 21/100, Loss: 9.2140
Epoch 31/100, Loss: 8.4632
Epoch 41/100, Loss: 7.3742
Epoch 51/100, Loss: 6.1614
Epoch 61/100, Loss: 4.8171
Epoch 71/100, Loss: 3.9106
Epoch 81/100, Loss: 3.3165
Epoch 91/100, Loss: 2.8881


In [62]:
mlp.loss

array([[15.21523039],
       [13.48855947],
       [12.54538391],
       [11.84098091],
       [11.49623096],
       [11.18797977],
       [11.10737227],
       [10.68867117],
       [10.75512428],
       [10.49354465],
       [10.31037003],
       [10.15610178],
       [ 9.97409187],
       [ 9.93743856],
       [ 9.88742857],
       [ 9.66304508],
       [ 9.53116878],
       [ 9.24315332],
       [ 9.50111403],
       [ 9.18220717],
       [ 9.21403813],
       [ 9.02749481],
       [ 9.02609293],
       [ 8.91917904],
       [ 8.78263556],
       [ 8.75563261],
       [ 8.55130505],
       [ 8.63630485],
       [ 8.41981368],
       [ 8.62965541],
       [ 8.4632166 ],
       [ 8.35534938],
       [ 8.24003727],
       [ 8.15382079],
       [ 8.04375002],
       [ 7.93632471],
       [ 7.82244011],
       [ 7.72476661],
       [ 7.60292927],
       [ 7.52107308],
       [ 7.3742165 ],
       [ 7.29143192],
       [ 7.15984279],
       [ 7.04506432],
       [ 6.91291764],
       [ 6

In [63]:
loss_mat = np.matrix(mlp.loss)

with open('100x95-500;500-0.001-100-ReLUloss.txt','wb') as f:
    for line in loss_mat:
        np.savetxt(f, line, fmt='%.5f')

In [59]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with ReLU and layers [9500, 500, 500, 4], lr = 0.01, epochs = 100:")
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))


MLP with ReLU and layers [9500, 500, 500, 4], lr = 0.01, epochs = 100:
Test Accuracy: 48.44%
Accuracy: 0.4844001485700136
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.39      0.48      5163
           1       0.39      0.47      0.42      4456
           2       0.43      0.51      0.47      3945
           3       0.60      0.65      0.63      2590

    accuracy                           0.48     16154
   macro avg       0.51      0.51      0.50     16154
weighted avg       0.51      0.48      0.49     16154

Confusion Matrix:
 [[2012 1858  974  319]
 [ 793 2099 1179  385]
 [ 343 1179 2018  405]
 [ 109  289  496 1696]]


In [61]:
# Evaluate accuracy on test set
predictions = mlp.predict(X_train)
y_test_labels = np.argmax(y_train, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print("\nMLP with ReLU and layers [9500, 500, 500, 4], lr = 0.01, epochs = 100:")
print(f'Train Accuracy: {accuracy * 100:.2f}%')
print("Accuracy:", accuracy_score(y_test_labels, predictions))
print("Classification Report:\n", classification_report(y_test_labels, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, predictions))


MLP with ReLU and layers [9500, 500, 500, 4], lr = 0.01, epochs = 100:
Train Accuracy: 50.07%
Accuracy: 0.500660338423442
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.40      0.49      7637
           1       0.40      0.50      0.44      6744
           2       0.44      0.51      0.47      5911
           3       0.66      0.69      0.68      3938

    accuracy                           0.50     24230
   macro avg       0.53      0.52      0.52     24230
weighted avg       0.52      0.50      0.50     24230

Confusion Matrix:
 [[3050 2734 1485  368]
 [1177 3346 1725  496]
 [ 538 1824 3002  547]
 [ 129  455  621 2733]]


In [25]:
print(mlp.loss)

[]
